In [1]:
!pip install geopy


In [8]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv("한국전력공사_부산시 전기차 충전소 충전량_20220331.csv", encoding="utf-8")  # 파일 인코딩을 utf-8-sig로 지정

# "주소" 필드 선택 후 리스트로 변환
address = df["주소"].dropna().tolist()  # NaN 값 제거 후 리스트로 변환

# 결과 출력
print(address)


['부산광역시 부산진구 가야대로 702', '부산광역시 부산진구 가야대로 702', '부산광역시 부산진구 거제대로 37', '부산광역시 부산진구 거제대로 37', '부산광역시 부산진구 초연로 11', '부산광역시 부산진구 초연로 11', '부산광역시 부산진구 가야대로 702', '부산광역시 부산진구 가야대로 702', '부산광역시 부산진구 가야대로 702', '부산광역시 부산진구 중앙대로666번길 43 (부전동)', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 중앙대로 666번길 43', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 시민공원로 30', '부산광역시 부산진구 가야대로 569 (가야동, 가야케이티 이-편한세상)', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 전포대로 189 (전포동)', '부산광역시 부산진구 전포대로 189 (전포동)', '부산광역시 부산진구 동성로 143', '부산광역시 부산진구 초연로 11', '부산광역시 부산진구 중앙대로644번길 20', '부산광역시 부산진구 중앙대로 666번길 43', '부산광역시 부산진구 중앙대로 666번길 43', '부산광역시 부산진구 초연로 11', '부산광역시 부산진구 중앙대로666번길 43 (부전동)', '부산광역시 부산진구 중앙대로666번길 43 (부전동)', '부산광역시 부산진구 중앙대로666번길 43 (부전동)', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 초연로 11', '부산광역시 부산진구 중앙대로 666번길 43', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 동천로 4', '부산광역시 부산진구 전포대로 1

In [ ]:
# 한줄씩 쓰기 작업,
import pandas as pd
import time
import csv
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

# Nominatim API 초기화
geolocator = Nominatim(user_agent="geo_converter")

# CSV 파일 로드
df = pd.read_csv("한국전력공사_부산시 전기차 충전소 충전량_20220331.csv", encoding="utf-8")

# "주소" 필드 선택 후 리스트 변환
addresses = df["주소"].dropna().tolist()

# 결과 저장 CSV 파일 경로
output_csv = "converted_addresses.csv"

# CSV 파일이 없는 경우 헤더 추가
if not os.path.exists(output_csv):
    with open(output_csv, mode="w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["주소", "위도", "경도"])  # 헤더 추가

# 주소 변환 함수 (예외 처리 + 1.5초 대기)
def get_lat_lon(address):
    try:
        location = geolocator.geocode(address, timeout=10)  # 10초 대기
        if location:
            lat, lon = location.latitude, location.longitude
        else:
            lat, lon = None, None
    except GeocoderTimedOut:
        print(f"⏳ 시간 초과: {address}, 다시 시도 중...")
        time.sleep(2)
        return get_lat_lon(address)  # 재시도

    print(f"✅ 주소 변환 완료: {address} → 위도: {lat}, 경도: {lon}")

    # CSV에 한 줄씩 추가 저장
    with open(output_csv, mode="a", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow([address, lat, lon])  # 한 줄씩 저장

    time.sleep(1.5)  # 1.5초 대기 (속도 제한 방지)
    return address, lat, lon

# 멀티스레딩 실행 (최대 5개 동시 실행)
with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_address = {executor.submit(get_lat_lon, addr): addr for addr in addresses}

    for future in as_completed(future_to_address):
        try:
            future.result()  # 각 변환 작업 실행
        except Exception as e:
            print(f"⚠️ 오류 발생: {e}")

print("🎉 모든 주소 변환 완료! 변환된 데이터가 'converted_addresses.csv'에 실시간으로 저장되었습니다.")


✅ 주소 변환 완료: 부산광역시 부산진구 거제대로 37 → 위도: 35.175996011961466, 경도: 129.067548200286
✅ 주소 변환 완료: 부산광역시 부산진구 가야대로 702 → 위도: 35.15408591044754, 경도: 129.03313149245102
✅ 주소 변환 완료: 부산광역시 부산진구 초연로 11 → 위도: 35.17369830770285, 경도: 129.04749603563283
✅ 주소 변환 완료: 부산광역시 부산진구 거제대로 37 → 위도: 35.175996011961466, 경도: 129.067548200286
✅ 주소 변환 완료: 부산광역시 부산진구 가야대로 702 → 위도: 35.15408591044754, 경도: 129.03313149245102
✅ 주소 변환 완료: 부산광역시 부산진구 초연로 11 → 위도: 35.17369830770285, 경도: 129.04749603563283
✅ 주소 변환 완료: 부산광역시 부산진구 가야대로 702 → 위도: 35.15408591044754, 경도: 129.03313149245102
✅ 주소 변환 완료: 부산광역시 부산진구 가야대로 702 → 위도: 35.15408591044754, 경도: 129.03313149245102
✅ 주소 변환 완료: 부산광역시 부산진구 가야대로 702 → 위도: 35.15408591044754, 경도: 129.03313149245102
✅ 주소 변환 완료: 부산광역시 부산진구 중앙대로666번길 43 (부전동) → 위도: None, 경도: None
✅ 주소 변환 완료: 부산광역시 부산진구 동천로 4 → 위도: 35.1574543, 경도: 129.0620397196934
✅ 주소 변환 완료: 부산광역시 부산진구 동천로 4 → 위도: 35.1574543, 경도: 129.0620397196934
✅ 주소 변환 완료: 부산광역시 부산진구 중앙대로 666번길 43 → 위도: None, 경도: None
✅ 주소 변환 완료: 부산광역시 부산진구 동천로 4 

In [ ]:
# 진행경과 표기.
import pandas as pd
import time
import csv
import os
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from concurrent.futures import ThreadPoolExecutor, as_completed

# Nominatim API 초기화
geolocator = Nominatim(user_agent="geo_converter")

# CSV 파일 로드
df = pd.read_csv("한국전력공사_부산시 전기차 충전소 충전량_20220331.csv", encoding="utf-8")

# "주소" 필드 선택 후 리스트 변환
addresses = df["주소"].dropna().tolist()

# 전체 주소 개수
total_count = len(addresses)
progress_count = 0  # 진행 개수 추적

# 결과 저장 CSV 파일 경로
output_csv = "converted_addresses.csv"

# CSV 파일이 없는 경우 헤더 추가
if not os.path.exists(output_csv):
    with open(output_csv, mode="w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["주소", "위도", "경도"])  # 헤더 추가

# 주소 변환 함수 (예외 처리 + 1.5초 대기 + 진행 상태 업데이트)
def get_lat_lon(address):
    global progress_count  # 진행 개수 업데이트를 위해 전역 변수 사용

    try:
        location = geolocator.geocode(address, timeout=10)  # 10초 대기
        if location:
            lat, lon = location.latitude, location.longitude
        else:
            lat, lon = None, None
    except GeocoderTimedOut:
        print(f"⏳ 시간 초과: {address}, 다시 시도 중...")
        time.sleep(2)
        return get_lat_lon(address)  # 재시도

    # 변환 완료 후 CSV에 한 줄씩 추가 저장
    with open(output_csv, mode="a", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow([address, lat, lon])

    # 진행 개수 업데이트
    progress_count += 1
    progress_percent = (progress_count / total_count) * 100
    print(f"✅ 진행 상태: {progress_count}/{total_count} 완료 ({progress_percent:.2f}%)")

    time.sleep(1.5)  # 1.5초 대기 (속도 제한 방지)
    return address, lat, lon

# 멀티스레딩 실행 (최대 5개 동시 실행)
with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_address = {executor.submit(get_lat_lon, addr): addr for addr in addresses}

    for future in as_completed(future_to_address):
        try:
            future.result()  # 각 변환 작업 실행
        except Exception as e:
            print(f"⚠️ 오류 발생: {e}")

print("🎉 모든 주소 변환 완료! 변환된 데이터가 'converted_addresses.csv'에 실시간으로 저장되었습니다.")


✅ 진행 상태: 1/14904 완료 (0.01%)
✅ 진행 상태: 2/14904 완료 (0.01%)
✅ 진행 상태: 3/14904 완료 (0.02%)
✅ 진행 상태: 4/14904 완료 (0.03%)
✅ 진행 상태: 5/14904 완료 (0.03%)
✅ 진행 상태: 6/14904 완료 (0.04%)
✅ 진행 상태: 7/14904 완료 (0.05%)
✅ 진행 상태: 8/14904 완료 (0.05%)
✅ 진행 상태: 9/14904 완료 (0.06%)
✅ 진행 상태: 10/14904 완료 (0.07%)
✅ 진행 상태: 11/14904 완료 (0.07%)
✅ 진행 상태: 12/14904 완료 (0.08%)
✅ 진행 상태: 13/14904 완료 (0.09%)
✅ 진행 상태: 14/14904 완료 (0.09%)
✅ 진행 상태: 15/14904 완료 (0.10%)
✅ 진행 상태: 16/14904 완료 (0.11%)
✅ 진행 상태: 17/14904 완료 (0.11%)
✅ 진행 상태: 18/14904 완료 (0.12%)
✅ 진행 상태: 19/14904 완료 (0.13%)
✅ 진행 상태: 20/14904 완료 (0.13%)
✅ 진행 상태: 21/14904 완료 (0.14%)
✅ 진행 상태: 22/14904 완료 (0.15%)
✅ 진행 상태: 23/14904 완료 (0.15%)
✅ 진행 상태: 24/14904 완료 (0.16%)
✅ 진행 상태: 25/14904 완료 (0.17%)
✅ 진행 상태: 26/14904 완료 (0.17%)
✅ 진행 상태: 27/14904 완료 (0.18%)
✅ 진행 상태: 28/14904 완료 (0.19%)
✅ 진행 상태: 29/14904 완료 (0.19%)
✅ 진행 상태: 30/14904 완료 (0.20%)
✅ 진행 상태: 31/14904 완료 (0.21%)
✅ 진행 상태: 32/14904 완료 (0.21%)
✅ 진행 상태: 33/14904 완료 (0.22%)
✅ 진행 상태: 34/14904 완료 (0.23%)
✅ 진행 상태: 35/14904 완료 (0